<a href="https://colab.research.google.com/github/philipayazi/Disaster_Tweets/blob/master/Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
import pickle

# Getting back the objects:
with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)

## DL Classifier from W2V Features

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train =train_tweet_matrix
X_test =test_tweet_matrix
y_train =y_train_split
y_test =y_test_split

print('Train X and y', X_train.shape, y_train.shape)
print('Test X and y', X_test.shape, y_test.shape)

Train X and y (5329, 300) (5329,)
Test X and y (2284, 300) (2284,)


In [8]:
#---------------- No scaling gives better results
X_train_scaled =X_train
X_test_scaled =X_test

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
y_train_categorical.shape

(5329, 2)

### Create DL model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=300))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               30100     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 40,402
Trainable params: 40,402
Non-trainable params: 0
_________________________________________________________________


In [15]:
#fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,  #60,
    shuffle=True,
    verbose=2,
    validation_split=0.15
)

Train on 4529 samples, validate on 800 samples
Epoch 1/10
4529/4529 - 3s - loss: 0.5059 - accuracy: 0.7587 - val_loss: 0.4669 - val_accuracy: 0.7912
Epoch 2/10
4529/4529 - 1s - loss: 0.4166 - accuracy: 0.8198 - val_loss: 0.4565 - val_accuracy: 0.7987
Epoch 3/10
4529/4529 - 1s - loss: 0.3836 - accuracy: 0.8337 - val_loss: 0.4540 - val_accuracy: 0.8037
Epoch 4/10
4529/4529 - 1s - loss: 0.3512 - accuracy: 0.8518 - val_loss: 0.4746 - val_accuracy: 0.7900
Epoch 5/10
4529/4529 - 1s - loss: 0.3205 - accuracy: 0.8664 - val_loss: 0.5007 - val_accuracy: 0.7862
Epoch 6/10
4529/4529 - 1s - loss: 0.2830 - accuracy: 0.8814 - val_loss: 0.5230 - val_accuracy: 0.7912
Epoch 7/10
4529/4529 - 1s - loss: 0.2520 - accuracy: 0.8973 - val_loss: 0.5908 - val_accuracy: 0.8000
Epoch 8/10
4529/4529 - 0s - loss: 0.2163 - accuracy: 0.9181 - val_loss: 0.5977 - val_accuracy: 0.7837
Epoch 9/10
4529/4529 - 1s - loss: 0.1800 - accuracy: 0.9293 - val_loss: 0.6782 - val_accuracy: 0.7937
Epoch 10/10
4529/4529 - 1s - loss: 

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=3)
print(
    f"RESULTS - Loss: {model_loss}, Accuracy: {model_accuracy}")

RESULTS - Loss: 0.6256960444609045, Accuracy: 0.7854641079902649


In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [18]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 0 0 0 0]
Actual Labels: [1, 0, 1, 0, 0]


## Hyperas ... for hyperparams optimization

In [21]:
def data():
    with open('objs.pkl', 'rb') as f:  
        test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)

    X_train =train_tweet_matrix
    X_test =test_tweet_matrix
    
    
    # Step 1: Label-encode data set
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train_split)
    encoded_y_train = label_encoder.transform(y_train_split)
    encoded_y_test = label_encoder.transform(y_test_split)

    # Step 2: Convert encoded labels to one-hot-encoding
    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)

    y_train =y_train_categorical
    y_test =y_test_categorical 
    
    X_scaler = StandardScaler().fit(X_train)
    
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    y_train_scaled = y_train
    y_test_scaled = y_test

    x_train = X_train_scaled
    x_test = X_test_scaled
    y_train = y_train_scaled
    y_test = y_test_scaled
   
    return x_train, y_train, x_test, y_test

In [22]:
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import backend as K
from keras.utils import print_summary
          
              
def create_model(x_train, y_train, x_test, y_test):
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    model= Sequential() 
    model.add(Dense( {{choice([50,100,500])}} , activation='relu', input_dim=300))
    model.add(Dense( {{choice([50,100,500])}} , activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print_summary(model, line_length=None, positions=None, print_fn=None)
    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([10,50,100])}},
                      verbose=2,
                      validation_split =0.15)
    
    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy = score[1]
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

best_run, best_model= optim.minimize(model=create_model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=5,
                                     trials=Trials(),
                                     notebook_name='Disaster_Tweets_hyperasW2VDL')


>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import classification_report, confusion_matrix
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    import pickle
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential

 - 0s - loss: 0.1433 - accuracy: 0.9592 - val_loss: 0.5374 - val_accuracy: 0.7850                                                                

Epoch 7/100                                                                                                                                      
 - 0s - loss: 0.1139 - accuracy: 0.9686 - val_loss: 0.5732 - val_accuracy: 0.7825                                                                

Epoch 8/100                                                                                                                                      
 - 0s - loss: 0.1083 - accuracy: 0.9715 - val_loss: 0.6051 - val_accuracy: 0.7875                                                                

Epoch 9/100                                                                                                                                      
 - 0s - loss: 0.0928 - accuracy: 0.9715 - val_loss: 0.6393 - val_accuracy: 0.7875                                        

Epoch 34/100                                                                                                                                     
 - 0s - loss: 0.0400 - accuracy: 0.9850 - val_loss: 0.9254 - val_accuracy: 0.7887                                                                

Epoch 35/100                                                                                                                                     
 - 0s - loss: 0.0380 - accuracy: 0.9830 - val_loss: 0.9356 - val_accuracy: 0.7950                                                                

Epoch 36/100                                                                                                                                     
 - 0s - loss: 0.0384 - accuracy: 0.9826 - val_loss: 0.9828 - val_accuracy: 0.7850                                                                

Epoch 37/100                                                                                                             

 - 0s - loss: 0.0283 - accuracy: 0.9839 - val_loss: 1.1148 - val_accuracy: 0.7925                                                                

Epoch 62/100                                                                                                                                     
 - 0s - loss: 0.0296 - accuracy: 0.9828 - val_loss: 1.1310 - val_accuracy: 0.7987                                                                

Epoch 63/100                                                                                                                                     
 - 0s - loss: 0.0280 - accuracy: 0.9848 - val_loss: 1.0997 - val_accuracy: 0.7887                                                                

Epoch 64/100                                                                                                                                     
 - 0s - loss: 0.0263 - accuracy: 0.9859 - val_loss: 1.1647 - val_accuracy: 0.7900                                        

Epoch 89/100                                                                                                                                     
 - 0s - loss: 0.0246 - accuracy: 0.9837 - val_loss: 1.3439 - val_accuracy: 0.7875                                                                

Epoch 90/100                                                                                                                                     
 - 0s - loss: 0.0246 - accuracy: 0.9859 - val_loss: 1.3651 - val_accuracy: 0.7912                                                                

Epoch 91/100                                                                                                                                     
 - 0s - loss: 0.0252 - accuracy: 0.9848 - val_loss: 1.3916 - val_accuracy: 0.7975                                                                

Epoch 92/100                                                                                                             

...

Epoch 15/50                                                                                                                                      
 - 0s - loss: 0.0686 - accuracy: 0.9773 - val_loss: 0.8985 - val_accuracy: 0.7788                                                                

Epoch 16/50                                                                                                                                      
 - 0s - loss: 0.0696 - accuracy: 0.9775 - val_loss: 0.9143 - val_accuracy: 0.7825                                                                

Epoch 17/50                                                                                                                                      
 - 0s - loss: 0.0621 - accuracy: 0.9803 - val_loss: 0.9411 - val_accuracy: 0.7775                                                                

Epoch 18/50                                                                                                              

 - 0s - loss: 0.0359 - accuracy: 0.9832 - val_loss: 1.2390 - val_accuracy: 0.7563                                                                

Epoch 43/50                                                                                                                                      
 - 0s - loss: 0.0366 - accuracy: 0.9819 - val_loss: 1.3130 - val_accuracy: 0.7887                                                                

Epoch 44/50                                                                                                                                      
 - 0s - loss: 0.0385 - accuracy: 0.9819 - val_loss: 1.2673 - val_accuracy: 0.7812                                                                

Epoch 45/50                                                                                                                                      
 - 0s - loss: 0.0509 - accuracy: 0.9799 - val_loss: 1.2808 - val_accuracy: 0.7812                                        

 - 0s - loss: 0.1277 - accuracy: 0.9609 - val_loss: 0.6285 - val_accuracy: 0.7738                                                                

Epoch 11/50                                                                                                                                      
 - 0s - loss: 0.1089 - accuracy: 0.9684 - val_loss: 0.6927 - val_accuracy: 0.7725                                                                

Epoch 12/50                                                                                                                                      
 - 0s - loss: 0.1019 - accuracy: 0.9702 - val_loss: 0.7146 - val_accuracy: 0.7788                                                                

Epoch 13/50                                                                                                                                      
 - 0s - loss: 0.0886 - accuracy: 0.9753 - val_loss: 0.7455 - val_accuracy: 0.7775                                        

Epoch 38/50                                                                                                                                      
 - 0s - loss: 0.0421 - accuracy: 0.9819 - val_loss: 1.0143 - val_accuracy: 0.7650                                                                

Epoch 39/50                                                                                                                                      
 - 0s - loss: 0.0416 - accuracy: 0.9828 - val_loss: 1.0447 - val_accuracy: 0.7725                                                                

Epoch 40/50                                                                                                                                      
 - 0s - loss: 0.0433 - accuracy: 0.9817 - val_loss: 1.0209 - val_accuracy: 0.7900                                                                

Epoch 41/50                                                                                                              

Epoch 6/10                                                                                                                                       
 - 1s - loss: 0.1327 - accuracy: 0.9607 - val_loss: 0.6686 - val_accuracy: 0.7925                                                                

Epoch 7/10                                                                                                                                       
 - 1s - loss: 0.1050 - accuracy: 0.9680 - val_loss: 0.6899 - val_accuracy: 0.7975                                                                

Epoch 8/10                                                                                                                                       
 - 1s - loss: 0.1063 - accuracy: 0.9700 - val_loss: 0.7171 - val_accuracy: 0.7638                                                                

Epoch 9/10                                                                                                               